# LeNet

LeNet是Yann LeCun在1998年设计并提出的，是早期卷积神经网路中最有代表性的实验系统之一，基于此神经网络模型的手写数字识别系统LeNet-5在20世纪90年代被广泛用于银行手写数字的识别系统。本文首先，以该模型为基础，对mnist数据集进行测试;然后，将模型用于车牌识别。

## 1 mnist样本字库的图片转换

有时为了方便研究和分析样本训练数据或测试样本数据，需要将测试样本库文件转换成图片，下面使用Pyhon脚本完成测试样本库格式转换成bmp格式图片。

In [1]:
# load library
import numpy as np
import struct
import matplotlib.pyplot as plt
import Image
import os

In [2]:
data_root="/home/ericdoug/mydev/caffe/data/mnist/"
filename = "t10k-images-idx3-ubyte"  # 压缩格式的测试样本库
raw_file = os.path.join(data_root, filename)

In [3]:
binfile = open(raw_file, 'rb')
buf = binfile.read()

In [4]:
index = 0
magic, numImages, numRows, numColumns = struct.unpack_from('>IIII', buf, index) 
index += struct.calcsize('>IIII')

In [5]:
print index, numImages, numRows, numColumns

16 10000 28 28


In [6]:
for image in range(0, numImages-1):
    im = struct.unpack_from('>784B', buf, index)
    index += struct.calcsize('>784B')
    
    im = np.array(im,dtype='uint8')
    im = im.reshape(28,28)
    
    im = Image.fromarray(im)
    
    im.save('../datas/mnist_test/train_%s.bmp'%image, 'bmp')

## 2. mnist手写测试

本节我们用手写的数字输入到网络进行分类识别、手写数字的图片必须满足以下条件：

>1.必须是256位黑白色

>2.必须是黑底白字

>3.像素大小必须是$28 \times 28$

>4.数字在图片中间，上下左右没有过多的空白。

In [8]:
# load library
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import caffe

In [9]:
model_file = '/home/ericdoug/mydev/caffe/examples/mnist/lenet.prototxt'
pretrained_caffemodel = '/home/ericdoug/mydev/caffe/examples/mnist/lenet_iter_10000.caffemodel'
image_file = '../datas/mnist_test/train_0.bmp'

In [ ]:
input_image = caffe.io.load_image(image_file, color=False)
net = caffe.Classifier(model_file, pretrained_caffemodel)
prediction = net.predict([input_image], oversample = False)
caffe.set_mode_gpu()
print 